In [87]:
import pandas as pd
import os
from pathlib import Path

In [89]:
# Пути к файлам
input_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\raw_n1"
output_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed"
output_file = os.path.join(output_folder, "combined_financial_data.xlsx")

In [91]:
# Создаем папку для результатов, если ее нет
os.makedirs(output_folder, exist_ok=True)

In [93]:
expected_columns = [
    'regnum',
    'namefo',
    'date_registration/reestr/licence',
    'inn',
    'ogrn',
    'address',
    'typefo'
]

all_data = []
sheet_stats = []  # Для сбора статистики по листам

In [95]:
# Обрабатываем все файлы Excel в папке
for file_name in os.listdir(input_folder):
    if file_name.endswith(('.xlsx', '.xls')):
        file_path = os.path.join(input_folder, file_name)
        date_of_relevance = Path(file_path).stem
        
        print(f"\nОбработка файла: {file_name} (дата: {date_of_relevance})")
        
        try:
            xls = pd.ExcelFile(file_path)

            # Обрабатываем каждый лист в файле
            for sheet_name in xls.sheet_names:
                try:
                    print(f"  Обработка листа: {sheet_name}...", end=" ")

                     # Читаем данные, нормализуем названия столбцов
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    df.columns = df.columns.str.strip().str.lower()
                    
                    # Добавляем отсутствующие столбцы
                    for col in expected_columns:
                        if col not in df.columns:
                            df[col] = None
                    
                    df = df[expected_columns].copy()
                    df['date_of_relevance'] = date_of_relevance
                    df['source_sheet'] = f"{file_name}::{sheet_name}" # Добавляем источник данных

                    # Сохраняем данные и статистику
                    records_count = len(df)
                    all_data.append(df)
                    sheet_stats.append({
                        'File': file_name,
                        'Sheet': sheet_name,
                        'Date': date_of_relevance,
                        'Records': records_count
                    })
                    
                    print(f"успешно! Записей: {records_count}")
                    
                except Exception as e:
                    print(f"ошибка! - {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"\nОшибка чтения файла {file_name}: {str(e)}")
            continue


Обработка файла: 01.01.2022.xlsx (дата: 01.01.2022)
  Обработка листа: КО... успешно! Записей: 370
  Обработка листа: Страховые организации... успешно! Записей: 147
  Обработка листа: ОВС... успешно! Записей: 16
  Обработка листа: Страховые брокеры... успешно! Записей: 59
  Обработка листа: Объединения... успешно! Записей: 20
  Обработка листа: Брокеры... успешно! Записей: 251
  Обработка листа: Дилеры... успешно! Записей: 279
  Обработка листа: Форекс-дилеры... успешно! Записей: 4
  Обработка листа: Управление ценными бумагами... успешно! Записей: 182
  Обработка листа: Депозитарии... успешно! Записей: 250
  Обработка листа: Ведение реестра ценных бумаг... успешно! Записей: 31
  Обработка листа: Инвестиционные советники... успешно! Записей: 126
  Обработка листа: Клиринг... успешно! Записей: 6
  Обработка листа: Биржи... успешно! Записей: 7
  Обработка листа: Операторы товарных поставок... успешно! Записей: 4
  Обработка листа: Репозитарии... успешно! Записей: 2
  Обработка листа: Це

In [99]:
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Функция для сохранения исходного формата числовых идентификаторов
    def preserve_format(value):
        if pd.isna(value):
            return None
        str_value = str(value).strip()
        # Удаляем .0 в конце, если есть (для случаев, когда число читалось как float)
        if '.' in str_value and str_value.endswith('0'):
            str_value = str_value.split('.')[0]
        return str_value
    
    # Обрабатываем столбцы с сохранением исходного формата
    if 'inn' in final_df.columns:
        final_df['inn'] = final_df['inn'].apply(preserve_format)
    if 'ogrn' in final_df.columns:
        final_df['ogrn'] = final_df['ogrn'].apply(preserve_format)
    
    try:
        # Сначала создаем временный DataFrame для записи
        temp_df = final_df.copy()
        
        # Создаем ExcelWriter с указанием файла
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Записываем основной лист
            temp_df.to_excel(writer, sheet_name='Combined Data', index=False)
            
            # Получаем объект листа для установки форматов
            workbook = writer.book
            worksheet = writer.sheets['Combined Data']
            
            # Устанавливаем текстовый формат для столбцов INN и OGRN
            for idx, col in enumerate(temp_df.columns):
                if col in ['inn', 'ogrn']:
                    # Получаем букву столбца (A, B, C...)
                    col_letter = chr(65 + idx) if idx < 26 else chr(64 + idx//26) + chr(65 + idx%26)
                    # Применяем текстовый формат ко всему столбцу
                    worksheet.column_dimensions[col_letter].number_format = '@'
            
            # Записываем дополнительные листы
            stats_df = pd.DataFrame(sheet_stats)
            
            # 1. Сводка по датам
            date_summary = stats_df.groupby('Date')['Records'].agg(['sum', 'count']).reset_index()
            date_summary.columns = ['Date', 'Total Records', 'Sheets Count']
            date_summary.to_excel(writer, sheet_name='Summary by Date', index=False)
            
            # 2. Детальная статистика по листам
            sheet_detail = stats_df[['Date', 'File', 'Sheet', 'Records']]
            sheet_detail.to_excel(writer, sheet_name='Sheet Details', index=False)
            
            # 3. Сводная таблица
            pivot_table = pd.pivot_table(stats_df, 
                                      values='Records',
                                      index='Date',
                                      columns='File',
                                      aggfunc='sum',
                                      fill_value=0)
            pivot_table.to_excel(writer, sheet_name='Pivot Date vs File')
            
        print(f"Файл успешно сохранен: {output_file}")
        print(f"Всего записей: {len(final_df)}")
        print(f"Уникальных дат: {final_df['date_of_relevance'].nunique()}")
        print(f"Обработано листов: {len(sheet_stats)}")
        
                
    except Exception as e:
        print(f"Ошибка при сохранении: {str(e)}")
else:
    print("Нет данных для обработки")

Файл успешно сохранен: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed\combined_financial_data.xlsx
Всего записей: 46912
Уникальных дат: 6
Обработано листов: 245


In [101]:
# Создаем датафрейм только с организациями из Москвы
moscow_pattern = r'(?i)(?:^|\s|г\.|\()(?:москва|мск)(?:\)|,|\s|$)'
mask = final_df['address'].str.contains(moscow_pattern, regex=True, na=False)
moscow_df = final_df[mask].copy()

In [103]:
# Путь для нового файла с московскими организациями
moscow_output_file = os.path.join(output_folder, "moscow_financial_data.xlsx")

In [105]:
if not moscow_df.empty:
    try:
        with pd.ExcelWriter(moscow_output_file, engine='openpyxl') as writer:
            # Основные данные по Москве
            moscow_df.to_excel(writer, sheet_name='Moscow Data', index=False)
            
            # Собираем статистику для московских данных
            moscow_stats = []
            
            # Группируем по дате актуальности
            for date, group in moscow_df.groupby('date_of_relevance'):
                moscow_stats.append({
                    'Date': date,
                    'Records': len(group),
                    'Unique INN': group['inn'].nunique(),
                    'Unique OGRN': group['ogrn'].nunique(),
                    'Unique Names': group['namefo'].nunique()
                })
            
            # Создаем датафрейм со статистикой
            moscow_stats_df = pd.DataFrame(moscow_stats)
            
            # 1. Общая статистика по датам
            moscow_stats_df.to_excel(writer, sheet_name='Summary by Date', index=False)
            
            # 2. Статистика по типам финансовых организаций
            type_stats = moscow_df['typefo'].value_counts().reset_index()
            type_stats.columns = ['Organization Type', 'Count']
            type_stats.to_excel(writer, sheet_name='Organization Types', index=False)
            
            # 3. Топ-10 организаций по количеству упоминаний
            top_orgs = moscow_df['namefo'].value_counts().head(10).reset_index()
            top_orgs.columns = ['Organization Name', 'Mentions Count']
            top_orgs.to_excel(writer, sheet_name='Top Organizations', index=False)
            
            # 4. Дополнительная проверка качества данных
            quality_check = pd.DataFrame({
                'Check': ['Total Moscow records', 
                         'Records with INN', 
                         'Records with OGRN',
                         'Records with both INN and OGRN'],
                'Count': [len(moscow_df),
                         moscow_df['inn'].notna().sum(),
                         moscow_df['ogrn'].notna().sum(),
                         (moscow_df['inn'].notna() & moscow_df['ogrn'].notna()).sum()]
            })
            quality_check.to_excel(writer, sheet_name='Data Quality', index=False)
            
        print(f"\nФайл с московскими организациями успешно сохранен: {moscow_output_file}")
        print(f"Всего записей по Москве: {len(moscow_df):,}")
        print(f"Уникальных дат для Москвы: {moscow_df['date_of_relevance'].nunique()}")
        print(f"Уникальных организаций по ИНН: {moscow_df['inn'].nunique():,}")
        print(f"Уникальных названий организаций: {moscow_df['namefo'].nunique():,}")
        
    except Exception as e:
        print(f"\nОшибка при сохранении московских данных: {str(e)}")
else:
    print("\nНе найдено организаций, относящихся к Москве")


Файл с московскими организациями успешно сохранен: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed\moscow_financial_data.xlsx
Всего записей по Москве: 13,365
Уникальных дат для Москвы: 6
Уникальных организаций по ИНН: 2,238
Уникальных названий организаций: 2,524


In [107]:
from dadata import Dadata
import pandas as pd
import time

In [109]:
# Настройки DaData
TOKEN = "my_token"
SECRET = "my_secret"
dadata = Dadata(TOKEN, SECRET)

In [111]:
# Выбираем уникальные организации
unique_orgs = moscow_df.drop_duplicates(
    subset=['inn', 'ogrn', 'namefo', 'address']
).reset_index(drop=True)

print(f"Найдено {len(unique_orgs)} уникальных организаций для обработки")

Найдено 4685 уникальных организаций для обработки


In [113]:
# Выбираем уникальные организации
unique_orgs = moscow_df.drop_duplicates(
    subset=['inn', 'ogrn', 'namefo']
).reset_index(drop=True)

print(f"Найдено {len(unique_orgs)} уникальных организаций для обработки")

Найдено 2741 уникальных организаций для обработки


In [115]:
# Выбираем уникальные организации
unique_orgs = moscow_df.drop_duplicates(
    subset=['inn', 'ogrn']
).reset_index(drop=True)

print(f"Найдено {len(unique_orgs)} уникальных организаций для обработки")

Найдено 2304 уникальных организаций для обработки
